In [1]:
import wandb
from datasets import load_dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling

2024-07-08 04:13:42.786927: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-08 04:13:42.787026: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-08 04:13:42.918959: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
wandb.init(project="gpt2-fine-tuning", name="fine-tune-gpt2")
print("done..")

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


done..


In [3]:
# Load tokenizer and add padding token if not present
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
print("done")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

done


In [4]:
# Defining function to load a subset of the dataset
def load_subset_dataset(dataset_path, num_samples=100):
    dataset = load_dataset('arrow', data_files={"train": dataset_path})
    return dataset['train'].shuffle().select(range(num_samples))
print("donee")

donee


In [5]:
# Tokenization function
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples["inputs_pretokenized"], padding="max_length", truncation=True, max_length=128)
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs
print("done")

done


In [6]:
# Load and preprocess a subset of the dataset
dataset_path = '/kaggle/input/intern-arrow/data-00000-of-00001.arrow'
tokenized_train_dataset = load_subset_dataset(dataset_path, num_samples=500)

# Tokenize dataset
tokenized_train_dataset = tokenized_train_dataset.map(tokenize_function, batched=True)

# Prepare data collator to handle data augmentation for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [7]:
# Load model and adjust for added tokens
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50258, 768)

In [8]:
# Load and preprocess a subset of the evaluation dataset
eval_dataset_path = '/kaggle/input/intern-arrow/data-00000-of-00001.arrow'
tokenized_eval_dataset = load_subset_dataset(eval_dataset_path, num_samples=10)

# Tokenize evaluation dataset
tokenized_eval_dataset = tokenized_eval_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [9]:
# Defining training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    logging_steps=500,
    save_steps=1000,
    evaluation_strategy="steps",
    logging_dir="./logs",
    report_to="wandb"  # Enable logging to Weights and Biases
)


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
# Initialize Trainer
# Initialize Trainer with both training and evaluation datasets
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,  # Specify the eval dataset here
    data_collator=data_collator,
)

# Train the model
trainer.train()




wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
500,3.109500,1.968977


TrainOutput(global_step=625, training_loss=2.8420236572265627, metrics={'train_runtime': 135.1042, 'train_samples_per_second': 18.504, 'train_steps_per_second': 4.626, 'total_flos': 163307520000000.0, 'train_loss': 2.8420236572265627, 'epoch': 5.0})

In [11]:
output_dir = "./fine-tuned-gpt2"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


('./fine-tuned-gpt2/tokenizer_config.json',
 './fine-tuned-gpt2/special_tokens_map.json',
 './fine-tuned-gpt2/vocab.json',
 './fine-tuned-gpt2/merges.txt',
 './fine-tuned-gpt2/added_tokens.json')

In [12]:
# Finish the Weights and Biases run
wandb.finish()

eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁█
train/global_step,▁▁█
train/grad_norm,▁
train/learning_rate,▁
train/loss,▁
eval/loss,1.96898
eval/runtime,0.2154
